# Topic Modeling TDS Posts with Latent Dirichlet Analysis

Based on the tutorial at https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
# !pip install gensim==3.8.3

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.ldamodel import LdaModel
from gensim.models.wrappers import LdaMallet

# NTLK stopwords
from nltk.corpus import stopwords

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

#saving models
import pickle 
import joblib

import warnings
warnings.filterwarnings("ignore")

In [3]:
#update stopwords with frequent irrelevant words identified during EDA
stop_words = stopwords.words('english')
stop_words.extend(['use', 'used', 'using', 'one', 'like', 'user', 'make', 'also', 'get', 'point', 'let', 'go'])

In [4]:
filename = '../data/cleaned_no_outliers.csv'

In [5]:
#read in the data
df = pd.read_csv(filename)

In [6]:
#remove non-text, non-Y columns
df = df.drop(columns=['date', 'responses', 'author_url',
       'story_url', 'reading_time (mins)', 'number_sections', 
       'number_paragraphs'])

#### Combine Title & Subtitle

In [7]:
df['combined_title'] = (df['title'] + ' ' + df['subtitle'])

In [8]:
df = df.drop(columns = ['title', 'subtitle']).reset_index(drop=True)

#### Remove Punctuation & Make Lowercase

In [9]:
text_cols = ['section_titles', 'paragraphs', 'combined_title']

In [10]:
for col in text_cols:
    df[col] = df[col].map(lambda x: re.sub("[''·,\.!?'-]", '', x).lower())

#### Remove the 'Min Read' Phrase at the Beginning of Each Paragraph

In [11]:
#remove all text up to and including 'min read' in the beginning of the paragraphs
df['paragraphs'] = df['paragraphs'].map(lambda x: (x[x.index('min read')+9:] if (x.count('min read') > 0) else x))
                     

#### Remove the Brackets Surrounding The Section Titles

In [12]:
#remove brackets surrounding the section titles
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("[",""))
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("]",""))

#### Combine Text Columns

In [13]:
df['text'] = df['combined_title'] + df['section_titles'] + df['paragraphs']

In [14]:
df = df.drop(columns = ['section_titles', 'paragraphs', 'combined_title'])

In [15]:
# Convert to list
data = df['text'].values.tolist()

#### Tokenize Words and Clean Up Text

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

data_words = list(sent_to_words(data))

#### Bigram and Trigram Models

In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#### Remove Stop Words 

In [19]:
data_words_nostops = remove_stopwords(data_words)

#### Initialize Spacy 'en' Model

In [20]:
#!spacy download en_core_web_lg

In [21]:
nlp = spacy.load('en_core_web_lg')

In [22]:
# remove exception from rule-based exceptions - not a fan of "datum" showing up in my corpus along with data....
nlp.get_pipe('lemmatizer').lookups.get_table("lemma_exc")["noun"]["data"] = ["data"]

#### Lemmatize

In [23]:
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','PROPN'])

#### Form Bigrams & Trigrams

In [24]:
data_words_bigrams = make_bigrams(data_lemmatized)

In [25]:
data_words_trigrams = make_trigrams(data_lemmatized)

### Trigram Modeling

#### Create the Dictionary & Corpus Needed for Topic Modeling

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_trigrams)

# Create Corpus
texts = data_words_trigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

### Build LDA Mallet Model

In [27]:
#start with 10 topics & fit model
mallet_path = '/Users/rachelinsler/dsi/downloads/mallet-2.0.8/bin/mallet' # update this path
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

#### Evaluate with Coherence Score & Visualization

In [28]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_words_trigrams, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
#display coherence score
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.484577186937358


INTERPRET

In [29]:
#  https://stackoverflow.com/questions/54684552/issue-with-topic-word-distributions-after-malletmodel2ldamodel-in-gensim
def ldaMalletConvertToldaGen(mallet_model):
    model_gensim = LdaModel(id2word=mallet_model.id2word, num_topics=mallet_model.num_topics, alpha=mallet_model.alpha, eta=0, iterations=1000, gamma_threshold=0.001, dtype=np.float32)
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [30]:
converted_model = ldaMalletConvertToldaGen(ldamallet)
pyLDAvis.enable_notebook()
plot = gensimvis.prepare(converted_model, corpus, id2word, sort_topics=False)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, '../figures/LDAmallet_trigrams_TDS10.html')
plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.160113  0.095085       1        1  10.931713
1      0.052278  0.158957       2        1  10.329586
2      0.242671 -0.021553       3        1   8.485157
3     -0.114045  0.098903       4        1  10.879970
4      0.006947 -0.241288       5        1  11.118851
5      0.260848  0.085396       6        1   9.131280
6      0.035753 -0.208050       7        1  12.186818
7     -0.095084  0.116110       8        1   9.322505
8     -0.177403 -0.135170       9        1   9.680532
9     -0.051853  0.051610      10        1   7.933589, topic_info=        Term           Freq          Total Category  logprob  loglift
241    model   66338.000000   66338.000000  Default  30.0000  30.0000
80      data  126417.000000  126417.000000  Default  29.0000  29.0000
50      code   26566.000000   26566.000000  Default  28.0000  28.0000
939    image   19633.000000   19633.000000  Default  27.0000  27.0000
289   python   19996.000000   19996.000000  Default  26.0000  26.0000
..       ...            ...            ...      ...      ...      ...
793  similar    1919.852981    6995.215082  Topic10  -5.3839   1.2411
760   result    2230.620618   18162.458859  Topic10  -5.2338   0.4370
150     give    2168.467130   16028.666484  Topic10  -5.2621   0.5337
142   follow    1828.924632   19912.576870  Topic10  -5.4324   0.1464
252   number    1721.882610   19512.361994  Topic10  -5.4927   0.1064

[978 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
424      10  0.977419      abcd
6855      7  1.000222  academia
1553      2  0.138882    access
1553      3  0.542311    access
1553      5  0.182804    access
...     ...       ...       ...
421       5  0.120965      year
421       7  0.318613      year
421       9  0.517811      year
1817      1  0.877352     ymean
2918      3  0.878788   ⁴define

[1938 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

#### Compute Optimal Number of Topics Based on Coherence

In [33]:
# an adapted version of a function to compute coherence values over a range of total topic numbers
# https://github.com/RaRe-Technologies/gensim/issues/2500

def compute_coherence_values(dictionary, all_corpus, texts, limit, start=2, step=2):
    coherence_values = []
    model_list = []
    num_topics_list = []

    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=all_corpus, num_topics=num_topics, id2word=dictionary,
                                         random_seed=42)
        #model = gensim.models.ldamodel.LdaModel(corpus=all_corpus,num_topics=num_topics,id2word=dictionary,eval_every=1,
        #                                        alpha='auto',random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        num_topics_list.append(num_topics)
        joblib.dump(model,f'ldamallet_model{num_topics}_tri.pkl')
        joblib.dump(id2word,f'id2word_dictionary_mallet{num_topics}_tri.pkl')

    return model_list, coherence_values, num_topics_list

In [ ]:
# calculate coherence values over the specific range of numbers of topics

model_list, coherence_values, num_topics_list = compute_coherence_values(dictionary=id2word,all_corpus=corpus,
                                                                         texts=data_words_trigrams,start=4,limit=30, step=2)

#store the results in a dataframe
model_values_df = pd.DataFrame({'model_list':model_list,'coherence_values':coherence_values,'num_topics':num_topics_list})

#find the number of topics that corresponds to the maximum coherence value
optimal_num_topics = model_values_df.loc[model_values_df['coherence_values'].idxmax()]['num_topics']

#save this as as the optimal model
optimal_model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=optimal_num_topics, id2word=id2word, random_seed=42)

In [ ]:
#save optimal models
joblib.dump(id2word,'id2word_dictionary_mallet_tri.pkl')
joblib.dump(optimal_model,'optimal_ldamallet_model_tri.pkl')

In [ ]:
# Show coherence values graph
limit=30; start=4; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.xticks(ticks=range(4,30,2))
plt.legend(("coherence_values"), loc='best')
plt.show()

### Create & Visualize Models with Optimal Number of Topics

INTERPRET

#### LDA Mallet Model: 10 Topics

In [59]:
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word,
                     iterations=2000,
                      topic_threshold=0.1,
                      random_seed=42,)

In [60]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48537638658319143


In [62]:
#convert and visualize
converted_model = ldaMalletConvertToldaGen(ldamallet)
pyLDAvis.enable_notebook()
plot = gensimvis.prepare(converted_model, corpus, id2word, sort_topics=False)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, '../figures/LDA_mallet_10.html')
plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.144218  0.096118       1        1  10.928875
1      0.155923  0.023021       2        1   9.501594
2     -0.113035 -0.278429       3        1  10.092524
3      0.000891  0.154164       4        1   9.472273
4      0.174075  0.064134       5        1  10.906809
5      0.114394 -0.204751       6        1  10.147244
6     -0.219390  0.173667       7        1  10.401691
7      0.061681 -0.004504       8        1   7.631174
8     -0.182364  0.035200       9        1  12.178637
9     -0.136393 -0.058619      10        1   8.739180, topic_info=          Term           Freq          Total Category  logprob  loglift
80        data  127175.000000  127175.000000  Default  30.0000  30.0000
242      model   66438.000000   66438.000000  Default  29.0000  29.0000
940      image   19577.000000   19577.000000  Default  28.0000  28.0000
933   function   29296.000000   29296.000000  Default  27.0000  27.0000
322    science   17472.000000   17472.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
384       time    2705.145671   29292.716442  Topic10  -5.1421   0.0552
1419    google    1655.046934    5117.162185  Topic10  -5.6334   1.3086
906     create    2063.672318   24751.155518  Topic10  -5.4127  -0.0470
1695     build    1842.238460   12195.797898  Topic10  -5.5262   0.5472
331        set    1821.692953   19682.701450  Topic10  -5.5375   0.0574

[962 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1124      2  0.170547  accuracy
1124      5  0.829415  accuracy
2300      1  0.576807    action
2300      3  0.204105    action
2300      7  0.219322    action
...     ...       ...       ...
420       4  0.486376      year
420       7  0.157469      year
420       9  0.329398      year
420      10  0.026715      year
1343      5  1.062431    youden

[1900 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

#### LDA Mallet Model: 16 Topics

In [ ]:
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=16, id2word=id2word,
                     iterations=2000,
                      topic_threshold=0.1,
                      random_seed=42,)

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#convert and visualize
converted_model = ldaMalletConvertToldaGen(ldamallet)
pyLDAvis.enable_notebook()
plot = gensimvis.prepare(converted_model, corpus, id2word, sort_topics=False)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, '../figures/LDA_mallet_16.html')
plot

#### LDA Mallet Model: 22 Topics

In [ ]:
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=22, id2word=id2word,
                     iterations=2000,
                      topic_threshold=0.1,
                      random_seed=42,)

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#convert and visualize
converted_model = ldaMalletConvertToldaGen(ldamallet)
pyLDAvis.enable_notebook()
plot = gensimvis.prepare(converted_model, corpus, id2word, sort_topics=False)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, '../figures/LDA_mallet_22.html')
plot